In [28]:
# This script shows you how to make a submission using a few
# useful Python libraries.
# It gets a public leaderboard score of 0.76077.
# Maybe you can tweak it and do better...?

import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV

In [4]:
# Load the data
train_df = pd.read_csv('data/train.csv', header=0)
train_df = train_df.drop('id', axis=1)
test_df = pd.read_csv('data/test_data.csv', header=0)
train_df

,fac_1,fac_2,fac_3,fac_4,fac_5,fac_6,fac_7,fac_8,poi_1,poi_2,poi_3,size,price_monthly,room_count,total_call,gender
0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1778.0,10038.0,4106.0,9.0000,1500000.0,6.0,72,campur
1,1.0,1.0,0.0,1.0,1.0,1.0,0.0,NaN,4548.0,9332.0,6867.0,12.0000,1500000.0,30.0,56,campur
2,1.0,NaN,1.0,1.0,1.0,1.0,0.0,1.0,5174.0,9021.0,3693.0,12.0000,1600000.0,20.0,109,campur
3,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1490.0,8954.0,2139.0,8.2500,1500000.0,15.0,54,campur
4,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1688.0,8851.0,2145.0,14.8500,2100000.0,10.0,19,campur
5,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1533.0,8715.0,3533.0,9.0000,1350000.0,7.0,5,putri
6,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,969.0,9079.0,2727.0,9.6250,1300000.0,20.0,20,putri
7,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,5200.0,8910.0,3762.0,10.5000,1000000.0,11.0,25,putri
8,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,4835.0,8490.0,3618.0,20.0000,4000000.0,8.0,4,putri
9,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,2880.0,8698.0,2210.0,9.0000,1950000.0,25.0,24,campur


In [5]:
# We'll impute missing values using the median for numeric columns and the most
# common value for string columns.
# This is based on some nice code by 'sveitser' at http://stackoverflow.com/a/25562948
from sklearn.base import TransformerMixin
class DataFrameImputer(TransformerMixin):
    def fit(self, X, y=None):
        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].median() for c in X],
            index=X.columns)
        return self
    def transform(self, X, y=None):
        return X.fillna(self.fill)

feature_columns_to_use = ['fac_1','fac_2','fac_3','fac_4','fac_5','fac_6','fac_7','fac_8','poi_1','poi_2'
                          ,'poi_3','size','price_monthly','room_count','total_call']
# nonnumeric_columns = ['Sex']

In [6]:
# Join the features from train and test together before imputing missing values,
# in case their distribution is slightly different
big_X = train_df[feature_columns_to_use].append(test_df[feature_columns_to_use])
big_X_imputed = DataFrameImputer().fit_transform(big_X)

In [8]:
# XGBoost doesn't (yet) handle categorical features automatically, so we need to change
# them to columns of integer values.
# See http://scikit-learn.org/stable/modules/preprocessing.html#preprocessing for more
# details and options
# le = LabelEncoder()
# for feature in nonnumeric_columns:
#     big_X_imputed[feature] = le.fit_transform(big_X_imputed[feature])

In [21]:
train_X = big_X_imputed[0:train_df.shape[0]].as_matrix()
test_X = big_X_imputed[train_df.shape[0]::].as_matrix()
train_y = train_df['gender']
X_train, X_test, Y_train, Y_test = train_test_split(train_X, train_y, test_size=0.3, random_state=101)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [49]:
# You can experiment with many other options here, using the same .fit() and .predict()
# methods; see http://scikit-learn.org
# This example uses the current build of XGBoost, from https://github.com/dmlc/xgboost
gbm = xgb.XGBClassifier(max_depth=11, n_estimators=2900, learning_rate=0.5).fit(train_X, train_y)
# gbm.score(X_test, Y_test)
predictions = gbm.predict(test_X)
predictions

array(['putri', 'putri', 'putri', 'putri', 'putri', 'putri', 'campur',
       'putri', 'campur', 'putri', 'putra', 'putri', 'putri', 'campur',
       'putra', 'putra', 'putri', 'putra', 'putra', 'campur', 'putri',
       'putri', 'putri', 'putri', 'putri', 'putri', 'putri', 'putri',
       'putra', 'putra', 'putra', 'putri', 'putri', 'putri', 'putri',
       'campur', 'putra', 'putri', 'putra', 'putra', 'putra', 'putri',
       'putra', 'campur', 'putri', 'putri', 'putri', 'campur', 'putri',
       'putri', 'putri', 'putra', 'putri', 'putra', 'putri', 'putri',
       'putra', 'campur', 'putra', 'campur', 'putri', 'putra', 'campur',
       'putri', 'campur', 'putri', 'putra', 'putra', 'putra', 'campur',
       'putri', 'campur', 'campur', 'campur', 'campur', 'campur',
       'campur', 'putra', 'putra', 'putra', 'putra', 'campur', 'putri',
       'putri', 'putri', 'putri', 'putri', 'putri', 'putri', 'putri',
       'putra', 'putri', 'putri', 'putri', 'putri', 'putri', 'putri',
       'pu

In [50]:
submission = pd.DataFrame({ 'id': test_df['id'],
                            'gender': predictions })
submission.to_csv("submission/24th_submission_XGBoost_transformer.csv", index=False)